In [ ]:
import os
import torch
import logging
from zipfile import ZipFile
from BudaOCR.Config import N_CHARSET
from huggingface_hub import snapshot_download
from BudaOCR.Modules import CRNNNetwork, OCRTrainer, WylieEncoder
from BudaOCR.Utils import shuffle_data, create_dir, build_data_paths
logging.getLogger().setLevel(logging.INFO)

torch.cuda.empty_cache()
print(torch.__version__)

In [2]:
charset = N_CHARSET
num_classes=len(charset)+1

In [ ]:
data_path = snapshot_download(repo_id="BDRC/Karmapa8", repo_type="dataset",  cache_dir="Datasets")

with ZipFile(f"{data_path}/data.zip", 'r') as zip:
    zip.extractall(f"{data_path}/Dataset")

dataset_path = f"{data_path}/Dataset"
image_paths, label_paths = build_data_paths(dataset_path)
image_paths, label_paths = shuffle_data(image_paths, label_paths)

print(f"Images: {len(image_paths)}, Labels: {len(label_paths)}")

output_dir = os.path.join("Output")
create_dir(output_dir)

In [8]:
dataset_path = "E:/Datasets/OCR/DbuMed/NEW/Drutsa-Batch31/batch31"
image_paths, label_paths = build_data_paths(dataset_path)
image_paths, label_paths = shuffle_data(image_paths, label_paths)

print(f"Images: {len(image_paths)}, Labels: {len(label_paths)}")

Images: 5946, Labels: 5946


In [9]:
wylie_encoder = WylieEncoder(N_CHARSET)
print(wylie_encoder.num_classes())

80


In [ ]:
output_dir = os.path.join(dataset_path, "Output")
create_dir(output_dir)

image_width = 3200
image_height = 100
num_classes = wylie_encoder.num_classes()

network = CRNNNetwork(image_width=image_width, image_height=image_height, num_classes=num_classes)
workers = 4

ocr_trainer = OCRTrainer(
    network=network,
    label_encoder=wylie_encoder,
    workers=workers, 
    image_width=image_width,
    image_height=image_height,
    batch_size=32, 
    output_dir=output_dir, 
    preload_labels=True
    )

ocr_trainer.init(image_paths, label_paths)

In [11]:
ocr_trainer.train(epochs=40, scheduler_start=32, check_cer=True)

Training network....


  0%|          | 0/148 [00:00<?, ?it/s]